In [40]:
import itertools
from collections import namedtuple

In [41]:
# card namedtuple type
Card = namedtuple('card', ['color', 'number', 'shape', 'shade'])

In [42]:
# characteristics
colors = ['purple', 'green', 'red']
numbers = ['one', 'two', 'three']
shapes = ['diamond', 'squiggle', 'oval']
shades = ['solid', 'striped', 'open']

In [43]:
# create all cards 
all_cards = {
    Card(color=color, number=number, shape=shape, shade=shade)
    for color in colors
    for number in numbers
    for shape in shapes
    for shade in shades
}

In [44]:
print(f"There are {len(all_cards)} cards in total.")

There are 81 cards in total.


In [45]:
all_cards_list = list(all_cards)
possible_sets = list(itertools.combinations(all_cards_list, 3))

In [50]:
print(f"There are {len(possible_sets):,} different 3 card combinations in total.")

There are 85,320 different 3 card combinations in total.


In [73]:
def check_if_valid(candidate_set):
    set_colors = set()
    set_numbers = set()
    set_shapes = set()
    set_shades = set()
    
    for card in candidate_set:
        set_colors.add(card.color)
        set_numbers.add(card.number)
        set_shapes.add(card.shape)
        set_shades.add(card.shade)

    set_totals = {len(set_colors), len(set_numbers), len(set_shapes), len(set_shades)}
    
    if set_totals in [{1},{3},{1,3}]:
        return True
    
    return False

In [74]:
valid_sets = set()
invalid_sets = set()

for candidate_set in possible_sets:
    if check_if_valid(candidate_set):
        valid_sets.add(candidate_set)
    else:
        invalid_sets.add(candidate_set)

In [75]:
assert len(valid_sets)+len(invalid_sets) == len(possible_sets)

In [76]:
print(f"There are {len(valid_sets):,} valid sets and {len(invalid_sets):,} invalid sets.")

There are 1,080 valid sets and 84,240 invalid sets.
